# Text Classfication using DeepLearning

https://github.com/krishnaik06/Fake-New-LSTM

In [ ]:
import pandas as pd
import numpy as np
import nltk, re
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional, Dropout
from tensorflow.keras.models import Sequential

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/rimanshu/Private_personal/master/Deep%20Learning/RNN/fake_news/train.csv/train.csv")

In [ ]:
!nvidia-smi

Thu Mar 11 19:56:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

In [ ]:
df =df.dropna()
df.shape

(18285, 5)

In [ ]:
X=df.drop('label', axis=1)
y=df['label']
X.shape,y.shape

((18285, 4), (18285,))

In [ ]:
tf.__version__

'2.4.1'

In [ ]:
## Vocab Size
voc_size = 5000

messages=X.copy()

messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[:10]

[[3878, 1858, 3569, 2351, 203, 4768, 455, 330, 3621, 4250],
 [4221, 4577, 3856, 3349, 2507, 4618, 3514],
 [3033, 2769, 383, 408],
 [2193, 3278, 2820, 4888, 458, 4442],
 [2061, 2507, 771, 3524, 2705, 4906, 2507, 2296, 1417, 627],
 [2158,
  3262,
  1566,
  3670,
  3750,
  1600,
  589,
  1385,
  2315,
  3940,
  22,
  269,
  1387,
  4186,
  3514],
 [4437, 1225, 3518, 254, 3665, 669, 101, 1441, 1364, 3787, 1070],
 [46, 3250, 2727, 19, 4731, 1422, 1600, 3935, 1364, 3787, 1070],
 [2131, 211, 1001, 4824, 140, 3245, 3349, 222, 1600, 3935],
 [2702, 4836, 2924, 4741, 291, 4663, 3791, 2488]]

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs[:4])

[[   0    0    0    0    0    0    0    0    0    0 3878 1858 3569 2351
   203 4768  455  330 3621 4250]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0 4221
  4577 3856 3349 2507 4618 3514]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 3033 2769  383  408]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
  2193 3278 2820 4888  458 4442]]


In [ ]:
## Creating model with simple LSTM network
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0159 - accuracy: 0.9947 - val_loss: 0.5557 - val_accuracy: 0.9031
Epoch 2/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0154 - accuracy: 0.9948 - val_loss: 0.5294 - val_accuracy: 0.9099
Epoch 3/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0125 - accuracy: 0.9959 - val_loss: 0.5318 - val_accuracy: 0.9100
Epoch 4/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0113 - accuracy: 0.9960 - val_loss: 0.5108 - val_accuracy: 0.9094
Epoch 5/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0119 - accuracy: 0.9961 - val_loss: 0.5868 - val_accuracy: 0.9057
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0135 - accuracy: 0.9963 - val_loss: 0.5925 - val_accuracy: 0.9059
Epoch 7/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0100 - accuracy: 0.9966 - val_loss: 0.6019 - val_accuracy: 0.9052
Epoch 

In [ ]:
## Creating model with drop out layer
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
## Creating model with bidectional layer.
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
y_pred=model.predict_classes(X_test)

confusion_matrix(y_test,y_pred)

accuracy_score(y_test,y_pred)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.9068765534382767

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.89      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

